# Neural Model in Keras

In [1]:
import os
import sys
import pickle as pkl

import keras
from keras import backend as K
from keras.models import Model
from keras.layers import Input, Embedding, Concatenate, Reshape, LSTM, Dense, Dropout
from keras.utils import multi_gpu_model, Sequence
from keras import optimizers
from keras.callbacks import TensorBoard

import numpy as np
import tensorflow as tf
from sklearn.metrics import confusion_matrix

sys.path.append(os.path.abspath('../preprocessing/'))
from data_parser import DataParser
from config import Config

Using TensorFlow backend.


## Data preparation

### Load and balance data

In [2]:
dp = DataParser(Config.embed_dim, Config.max_seq_len)
x_train_seq, x_train_syn, x_train_lex, y_train, embed_matrix = dp.load_data()

### Define Model

In [3]:
seq_input = Input(shape=(Config.max_seq_len,), dtype='int32', name='seq_input')
syn_input = Input(shape=(Config.max_seq_len,), dtype='float32', name='syn_input')
lex_input = Input(shape=(4082,), dtype='float32', name='lex_input')

seq_embeds = Embedding(embed_matrix.shape[0], Config.embed_dim, 
                       weights=[embed_matrix], input_length=Config.max_seq_len,
                       trainable=False,
                       embeddings_regularizer=keras.regularizers.l2(.01),
                       name='embedding_layer')(seq_input)

# syn_addons = Reshape((Config.max_seq_len, 1), name='reshape_layer')(syn_input)

time_series = seq_embeds
# time_series = Concatenate()([seq_embeds, syn_addons])

lstm_output = LSTM(Config.lstm_units, 
                   activation='relu',
                   name='lstm_layer')(time_series)

lex_feed = lex_input
for i in range(3):
    lex_feed = Dense(Config.lstm_units,
                 activation='relu',
                 kernel_regularizer=keras.regularizers.l2(.01),
                 name='lex_dense_layer_{}'.format(i))(lex_feed)
    lex_feed = Dropout(.01)(lex_feed)

concat = Concatenate()([lstm_output, lex_feed])

concat_feed = Dense(Config.lstm_units,
                    activation='relu',
                    kernel_regularizer=keras.regularizers.l2(.01),
                    name='concat_dense_layer_{}'.format(i))(concat)
concat_feed = Dropout(.01)(concat_feed)

predictions = Dense(Config.num_classes,
                    activation='softmax',
                    name='predictions')(concat_feed)

model = Model(inputs=[seq_input, syn_input, lex_input], outputs=predictions)

sgd = optimizers.SGD(lr=Config.learning_rate, decay=Config.learning_decay)
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['acc'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
lex_input (InputLayer)          (None, 4082)         0                                            
__________________________________________________________________________________________________
lex_dense_layer_0 (Dense)       (None, 5)            20415       lex_input[0][0]                  
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, 5)            0           lex_dense_layer_0[0][0]          
__________________________________________________________________________________________________
lex_dense_layer_1 (Dense)       (None, 5)            30          dropout_1[0][0]                  
__________________________________________________________________________________________________
seq_input 

### Run model

In [ ]:
run = 'fine'

tensorboard = TensorBoard(log_dir='./logs/{}'.format(run), histogram_freq=0,
                          write_graph=True, write_images=False)

for i in range(5):
    model.fit([x_train_seq, x_train_syn, x_train_lex], y_train, 
              batch_size=Config.batch_size, 
              epochs=Config.epochs, 
              validation_split=0.20,
              callbacks=[tensorboard])
    model.save('./models/{}_{}.h5'.format(run, i))

y_pred_test = model.predict_on_batch([x_train_seq, x_train_syn, x_train_lex])
print confusion_matrix(y_train.argmax(axis=-1), y_pred_test.argmax(axis=-1))

del model

Train on 1600 samples, validate on 400 samples
Epoch 1/10
1600/1600 [==============================] - 15s 9ms/step - loss: nan - acc: 0.1919 - val_loss: nan - val_acc: 0.1850
Epoch 2/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.1944 - val_loss: nan - val_acc: 0.1850
Epoch 3/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.1925 - val_loss: nan - val_acc: 0.1850
Epoch 4/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.2000 - val_loss: nan - val_acc: 0.1850
Epoch 5/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.1969 - val_loss: nan - val_acc: 0.1850
Epoch 6/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.1938 - val_loss: nan - val_acc: 0.1850
Epoch 7/10
1600/1600 [==============================] - 14s 9ms/step - loss: nan - acc: 0.1950 - val_loss: nan - val_acc: 0.1850
Epoch 8/10
1600/1600 [============================